In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
import pandas as pd
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import  RandomForestClassifier,RandomForestClassificationModel
import numpy as np
from pyspark.ml.linalg import *
from pyspark.ml.linalg import Vectors, VectorUDT
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
features_train = spark.sql("select * from VZW_SOI_PRD_TBLS.vzsoi_intl_call_base_tbl_v5")

In [ ]:
features_train.dtypes

In [ ]:
#https://arxiv.org/pdf/1904.07199.pdf
features=features_train.select('mtn','ivr_call_id','msg_allow_share_ind','data_allow_share_ind','voice_allow_share_ind','lst_session_mfapp_activate_tp_ind','lst_session_mfapp_billing_ind',
                               'lst_session_mfapp_cruise_ind','lst_session_mfapp_usage_ind','lst_session_mfapp_support_ind',
                               'lst_session_mfapp_can_mex_ind',
                               'lst_session_mfapp_tp_enquiry_ind','lst_session_mfapp_countries_ind',
                               'lst_session_mfapp_disable_tp_ind',
                               'mvo_activate_tp_ind','mvo_intl_billing_ind','mvo_tripplanner_ind',
                               'mvo_cruise_ind','mvo_tp_video_ind','mvo_usage_ind','mvo_faq_international_ind',
                               'mvo_device_support_ind','mvo_faq_general_ind','mvo_faq_outsideus_ind',
                               'mvo_settings_ind','mvo_can_mex_ind','mvo_unlimitedplans_ind',
                               'mvo_tp_enquiry_ind','mvo_last_pg_vst_time',
                               'mva_activate_tp_ind','mva_intl_billing_ind','mva_cruise_ind',
                               'mva_usage_ind','mva_device_support_ind',
                               'mva_can_mex_ind','mva_tp_enquiry_ind','mva_countries_ind','mva_disable_tp_ind',
                               'mva_devices_ind','mva_last_pg_vst_time',
                               'lst_session_activate_tp_ind','lst_session_billing_ind','lst_session_tripplanner_ind',
                               'lst_session_cruise_ind','lst_session_tp_video_vstd_ind','lst_session_usage_ind',
                               'lst_session_faq_international_ind','lst_session_device_support_ind',
                               'lst_session_faq_general_ind','lst_session_faq_outsideus_ind',
                               'lst_session_settings_ind',
                               'lst_session_can_mex_ind','lst_session_unlimitedplan_ind',
                               'lst_session_tp_enquiry_ind','lst_session_faq_prepaid_ind',
                               'pplan_eff_dt','access_amt','data_access_amt','bi_pplan_ctgry_desc',                              'bi_data_allow_tier_value','svc_prod_desc1','svc_prod_desc2','svc_prod_desc3',
                               'mtrans_ts1','mtrans_ts2','mtrans_ts3',
                               'max_svc_prod_eff_dt1','free_plan','line_type',
                               'tvl_pass_included','can_mex_free_plan','region_desc','call_start_tm_stmp','bill_cycle_dt',
                               'feat_amt','pymnt_recvd_amt','bill_bal_amt','bill_curr_amt','LATE_PYMNT_CHRG_AMT',
                               'bill_6_mth_avg','occ_reasons','trans_dt',
                               'agg_occ_value','occ_amt',
                               'prev_agg_occ_value','prev_feat_amt','prev_due_amt','prev_pymnt_recvd_amt',
                               'prev_bill_bal_amt','prev_late_pymnt_chrg_amt','prev_occ_amt','prev_bill_curr_amt','prev_trans_dt',
                               'total_data_allow','plan_data_allow','bonus_data_allow','tenure','call_ts1',
                               'call_ts2','call_ts3','call_ts4','call_ts5','call_ts6','call_ts7','call_ts8','call_ts9','call_ts10',
                               'primary1_topic','primary2_topic','primary3_topic','primary4_topic',
                               'primary5_topic','primary6_topic','primary7_topic','primary8_topic','primary9_topic','primary10_topic',
                               'tvl1','tvl2','tvl3','tvl4','tvl5','tvl6','tvl7','tvl8','tvl9','tvl10',
                               'ccd1','ccd2','ccd3','ccd4','ccd5','ccd6','ccd7','ccd8','ccd9','ccd10',
                               'primary_topic',
                               'prev_tvl_count','prev_call_count','pplan_bndl_type_cd',
                               'alert_type','MSG_TIMESTAMP')

In [ ]:
features=features.dropna(subset=['primary_topic'])


In [ ]:
filter_site_columns=['lst_session_mfapp_activate_tp_ind','lst_session_mfapp_billing_ind',
                               'lst_session_mfapp_cruise_ind','lst_session_mfapp_usage_ind','lst_session_mfapp_support_ind',
                               'lst_session_mfapp_can_mex_ind',
                               'lst_session_mfapp_tp_enquiry_ind','lst_session_mfapp_countries_ind',
                               'lst_session_mfapp_disable_tp_ind',
                               'mvo_activate_tp_ind','mvo_intl_billing_ind','mvo_tripplanner_ind',
                               'mvo_cruise_ind','mvo_tp_video_ind','mvo_usage_ind','mvo_faq_international_ind',
                               'mvo_device_support_ind','mvo_faq_general_ind','mvo_faq_outsideus_ind',
                               'mvo_settings_ind','mvo_can_mex_ind','mvo_unlimitedplans_ind',
                               'mvo_tp_enquiry_ind',
                               'mva_activate_tp_ind','mva_intl_billing_ind','mva_cruise_ind',
                               'mva_usage_ind','mva_device_support_ind',
                               'mva_can_mex_ind','mva_tp_enquiry_ind','mva_countries_ind','mva_disable_tp_ind',
                               'mva_devices_ind',
                               'lst_session_activate_tp_ind','lst_session_billing_ind','lst_session_tripplanner_ind',
                               'lst_session_cruise_ind','lst_session_tp_video_vstd_ind','lst_session_usage_ind',
                               'lst_session_faq_international_ind','lst_session_device_support_ind',
                               'lst_session_faq_general_ind','lst_session_faq_outsideus_ind',
                               'lst_session_settings_ind',
                               'lst_session_can_mex_ind','lst_session_unlimitedplan_ind',
                               'lst_session_tp_enquiry_ind','lst_session_faq_prepaid_ind']

In [ ]:
filter_real_time=['lst_session_mfapp_activate_tp_ind','lst_session_mfapp_billing_ind',
                               'lst_session_mfapp_cruise_ind','lst_session_mfapp_usage_ind','lst_session_mfapp_support_ind',
                               'lst_session_mfapp_can_mex_ind',
                               'lst_session_mfapp_tp_enquiry_ind','lst_session_mfapp_countries_ind',
                               'lst_session_mfapp_disable_tp_ind','lst_session_activate_tp_ind','lst_session_billing_ind','lst_session_tripplanner_ind',
                               'lst_session_cruise_ind','lst_session_tp_video_vstd_ind','lst_session_usage_ind',
                               'lst_session_faq_international_ind','lst_session_device_support_ind',
                               'lst_session_faq_general_ind','lst_session_faq_outsideus_ind',
                               'lst_session_settings_ind',
                               'lst_session_can_mex_ind','lst_session_unlimitedplan_ind',
                               'lst_session_tp_enquiry_ind','lst_session_faq_prepaid_ind']

In [ ]:
filter_batch_time=['mvo_activate_tp_ind','mvo_intl_billing_ind','mvo_tripplanner_ind',
                               'mvo_cruise_ind','mvo_tp_video_ind','mvo_usage_ind','mvo_faq_international_ind',
                               'mvo_device_support_ind','mvo_faq_general_ind','mvo_faq_outsideus_ind',
                               'mvo_settings_ind','mvo_can_mex_ind','mvo_unlimitedplans_ind',
                               'mvo_tp_enquiry_ind',
                               'mva_activate_tp_ind','mva_intl_billing_ind','mva_cruise_ind',
                               'mva_usage_ind','mva_device_support_ind',
                               'mva_can_mex_ind','mva_tp_enquiry_ind','mva_countries_ind','mva_disable_tp_ind',
                               'mva_devices_ind']

In [ ]:
for i in filter_site_columns:
    features = features.withColumn('{}'.format(i),when(col('{}'.format(i)).isNull(),0.0).otherwise(col('{}'.format(i))))
    features = features.withColumn('{}'.format(i), features['{}'.format(i)].cast(FloatType()))

In [ ]:
date_columns=['pplan_eff_dt','mtrans_ts1','mtrans_ts2','mtrans_ts3','max_svc_prod_eff_dt1',
             'bill_cycle_dt','trans_dt','prev_trans_dt',
             'call_ts1','call_ts2','call_ts3','call_ts4','call_ts5','call_ts6','call_ts7','call_ts8','call_ts9','call_ts10',
              'tvl1','tvl2','tvl3','tvl4','tvl5','tvl6','tvl7','tvl8','tvl9','tvl10',
              'MSG_TIMESTAMP','mvo_last_pg_vst_time','mva_last_pg_vst_time']

In [ ]:
features=features.drop_duplicates()

In [ ]:
date_diff=[-float("inf"),0.0,2.0,7.0,30.0,60.0,90.0,180.0,float("inf")]

In [ ]:
date_splits={}
date_bucketizer={}
for i in date_columns:
    features=features.withColumn('call_{}'.format(i),datediff('call_start_tm_stmp','{}'.format(i)))
    features=features.withColumn('call_{}'.format(i),when(col('call_{}'.format(i))<0,None).otherwise(col('call_{}'.format(i))))
    #date_splits[i]=[-float("inf")]+np.unique(features.approxQuantile('call_{}'.format(i),(date_diff),thsld)).tolist()+[float("inf")]
    date_bucketizer[i] = Bucketizer(splits=date_diff,inputCol="call_{}".format(i), outputCol="call_{}_buckets".format(i),handleInvalid="keep")
    features=date_bucketizer[i].transform(features)
    date_bucketizer[i].save('/user/kreddy01/call_intet_intl_prediction_experiment/model_object_02_26_2020/value_bucketizer_{}'.format(i))

In [ ]:
value_diff=[0.1,0.25,0.5,0.75,0.9]
thsld=0

In [ ]:
double_columns=['prev_tvl_count','prev_call_count','bi_data_allow_tier_value']

In [ ]:
features=features.withColumn('bill_shock',(col('bill_curr_amt')-col('bill_6_mth_avg'))/col('bill_6_mth_avg'))

In [ ]:
##Bill_30_day_amt, 90_day_amt and 180 _day_amt are not populated properly
float_columns=['access_amt', 'data_access_amt','feat_amt','pymnt_recvd_amt','bill_bal_amt','bill_curr_amt','bill_shock','LATE_PYMNT_CHRG_AMT',
                               'agg_occ_value','occ_amt',
                               'prev_agg_occ_value','prev_feat_amt','prev_due_amt','prev_pymnt_recvd_amt',
                               'prev_bill_bal_amt','prev_late_pymnt_chrg_amt','prev_occ_amt','prev_bill_curr_amt']
#payment revrs_amt is not considered becuase it rarely has values

In [ ]:
value_columns=float_columns+double_columns
for i in value_columns:
    if i in double_columns:
        features = features.withColumn('{}'.format(i), features['{}'.format(i)].cast(DoubleType()))
    else:
        features = features.withColumn('{}'.format(i), features['{}'.format(i)].cast(FloatType())) 

In [ ]:
#float_columns_ab=['bill_shock','agg_occ_value','prev_agg_occ_value','occ_amt','prev_occ_amt']
for i in float_columns:
    features=features.withColumn('{}'.format(i), when(col('{}'.format(i))<0.0,0.0).otherwise(col('{}'.format(i))))

In [ ]:
#Requires More Data to normalize these columns, but a good split value_diff
value_splits={}
value_bucketizer={}
value_columns=float_columns+double_columns
decimals_1_float=['feat_amt','pymnt_recvd_amt','bill_bal_amt','bill_curr_amt','agg_occ_value','prev_agg_occ_value','prev_feat_amt',
                 'prev_due_amt','prev_pymnt_recvd_amt','prev_bill_bal_amt','prev_bill_curr_amt']
for i in value_columns:
    value_splits[i]=[-float("inf")]+np.unique(features.approxQuantile('{}'.format(i),(value_diff),thsld)).tolist()+[float("inf")]
    if i in decimals_1_float:
        value_splits[i]=np.around(value_splits[i],decimals=-1)
    elif i=='bill_shock':
        value_splits[i]=np.around(value_splits[i],decimals=1)
    else:
        value_splits[i]=np.around(value_splits[i])
    value_splits[i]=np.unique(value_splits[i])
    value_bucketizer[i] = Bucketizer(splits=value_splits[i],inputCol="{}".format(i), outputCol="{}_buckets".format(i),handleInvalid="keep")
    features=value_bucketizer[i].transform(features)
    value_bucketizer[i].save('/user/kreddy01/call_intet_intl_prediction_experiment/model_object_02_26_2020/value_bucketizer_{}'.format(i))

In [ ]:
value_splits

In [ ]:
value_mod=['{}_buckets'.format(x) for x in value_columns]
date_mod=['call_{}_buckets'.format(x) for x in date_columns]
#value_date_splits={**value_splits,**date_splits}

In [ ]:
import pickle
f = open("value_splits.pkl","wb")
pickle.dump(value_splits,f)
f.close()
! hdfs dfs -put value_splits.pkl hdfs://polarisgq-nn1.gq.vcg.yahoo.com:8020/user/kreddy01/call_intet_intl_prediction_experiment/model_object_02_26_2020

In [ ]:
value_bucket_list={}
value_splits_list=list(value_splits.values())
for i,v in enumerate(value_mod):
    #max_bucket_list[i]=features.groupBy().max('{}'.format(i)).distinct().rdd.flatMap(lambda x: x).collect()[0]
    value_bucket_list[v]=len(value_splits_list[i])
    features=features.withColumn('{}'.format(v),when(col('{}'.format(v)).isNull(),value_bucket_list[v]).otherwise(col('{}'.format(v))))

In [ ]:
for i,v in enumerate(date_mod):
    #max_bucket_list[i]=features.groupBy().max('{}'.format(i)).distinct().rdd.flatMap(lambda x: x).collect()[0]
    date_bucket_max=len(date_diff)
    features=features.withColumn('{}'.format(v),when(col('{}'.format(v)).isNull(),date_bucket_max).otherwise(col('{}'.format(v))))

In [ ]:
#Documentation requried for categorical values
cat={}
cat['free_plan']=['free','paid']
cat['line_type']=['PRIMARY','SECONDARY']
cat['occ_reasons']=['data_usg_charges','General_reasons','International_charges','occ_trvlpass','ROAMING']
cat['tenure']=['1 YR to 2 YR','2 YR to 3 YR','3 YR+','Up to 1 Year']
cat['alert_type']=['BILL_SMS','Daypass_SMS','Data_SMS','International_SMS','TP_SMS']
for i in range(1,4):
    cat['svc_prod_desc{}'.format(i)]=["INT'L HIGH SPEED DATA","TRAVELPASS","INTL TRAVEL VOICE & DATA PAYGO","INCLUDE CANADA/MEXICO"]
for i in range(1,11):
    cat['primary{}_topic'.format(i)]=["BILLING","CRUISE","PHONE SETTINGS","PLAN & TRAVEL PASS"]
    cat['ccd{}'.format(i)]=['CAN','MEX','GBR','BHS','JAM','DOM','FRA','ITA','CSP','ABW','DEU','ESP']
cat['primary_topic']=["BILLING","CRUISE","PHONE SETTINGS","PLAN & TRAVEL PASS"]
cat['msg_allow_share_ind']=['Y','N']
cat['data_allow_share_ind']=['Y','N']
cat['voice_allow_share_ind']=['Y','N']

In [ ]:
features=features.withColumn('free_plan',lower(col('free_plan')))

In [ ]:
cat_columns=[*cat.keys()]

In [ ]:
fromlabelsModel={}
model_string={}
for i in cat_columns:
    features=features.withColumn('{}'.format(i),when(col('{}'.format(i)).isin(*cat[i])==False,None).otherwise(col('{}'.format(i))))
    fromlabelsModel[i] = StringIndexer(inputCol="{}".format(i), outputCol="indexed_{}".format(i), handleInvalid="keep",
    stringOrderType="alphabetAsc")
    model_string[i]=fromlabelsModel[i].fit(features)
    features = model_string[i].transform(features)
    model_string[i].save('/user/kreddy01/call_intet_intl_prediction_experiment/model_object_02_26_2020/model_string_{}'.format(i))

In [ ]:
cat_mod=['indexed_{}'.format(x) for x in cat_columns]

In [ ]:
total_mod=cat_mod+filter_batch_time+filter_real_time+date_mod+value_mod

In [ ]:
features_model=features.select(total_mod)

In [ ]:
features_model.dtypes

In [ ]:
ohe={}
model_ohe={}
for i in cat_mod:
    ohe[i]=OneHotEncoderEstimator(inputCols=['{}'.format(i)],outputCols=['{}_ohe'.format(i)],handleInvalid='error',dropLast=True)  
    model_ohe[i]=ohe[i].fit(features_model)
    features_model=model_ohe[i].transform(features_model)
    model_ohe[i].save('/user/kreddy01/call_intet_intl_prediction_experiment/model_object_02_26_2020/model_ohe_{}'.format(i))

In [ ]:
ohe_list = list(((filter(lambda x: ('ohe' in x), list(features_model.columns)))))

In [ ]:
ohe_list_split={}
udf_numpy=udf(lambda x : np.array(x).tolist(),ArrayType(FloatType()))
for i in ohe_list:
    features_model=features_model.withColumn('{}_numpy'.format(i),udf_numpy('{}'.format(i)))
    ohe_list_split[i]=features_model.select(size(features_model['{}_numpy'.format(i)])).toPandas()['size({}_numpy)'.format(i)][0]
    for j in range(ohe_list_split[i]):                                            
        features_model=features_model.withColumn('{}_numpy_{}'.format(i,j),features_model['{}_numpy'.format(i)][j])

In [ ]:
features_model.dtypes

In [ ]:
ohe_list_numpy = list(filter(lambda x: ('ohe_numpy_' in x), list(features_model.columns)))

In [ ]:
buckets_list = list(filter(lambda x: ('buckets' in x), list(features_model.columns)))

In [ ]:
batch_features=np.array(ohe_list_numpy)
batch_features=np.append(batch_features,'indexed_primary_topic')
batch_features=np.append(batch_features,filter_batch_time)
batch_features=np.append(batch_features,buckets_list)

In [ ]:
batch_features.tolist()

In [ ]:
all_features=np.append(batch_features,filter_real_time)

In [ ]:
features_train_pandas_all=features_model.select(*all_features).toPandas()

In [ ]:
train_pandas=features_train_pandas_all

In [ ]:
train_pandas['current_dt_1msg']=np.where(train_pandas['call_tvl1_buckets']<=1.0, 1.0, 0.0)

In [ ]:
output_one_hot=['indexed_primary_topic_ohe_numpy_0','indexed_primary_topic_ohe_numpy_1','indexed_primary_topic_ohe_numpy_2','indexed_primary_topic_ohe_numpy_3','indexed_primary_topic']
all_columns=train_pandas.columns
input_cols=np.delete(all_columns, [i for i,v in enumerate(all_columns) if v in output_one_hot])

In [ ]:
list(input_cols)

In [ ]:
y=train_pandas['indexed_primary_topic']

In [ ]:
X=train_pandas[list(input_cols)]

In [ ]:
np.unique(y,return_counts=True)

In [ ]:
y=y.replace(4.0,3.0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
x_train=X.head(1000000)
y_train=y.head(1000000)
clf = RandomForestClassifier(n_estimators=100,max_depth=20, random_state=0,class_weight='balanced')

In [ ]:
x_train_np=np.array(x_train)
y_train_np=np.array(y_train)

In [ ]:
clf.fit(x_train,y_train)

In [ ]:
D=pd.DataFrame(X.tail(200000))
D['label']=y.tail(200000)
trvl_once=D['call_tvl1_buckets']>1.0
D=D[trvl_once]

In [ ]:
confusion_matrix(np.array(y.tail(200000)),clf.predict(np.array(X.tail(200000))))

In [ ]:
confusion_matrix(np.array(D['label']),clf.predict(np.array(D.loc[:,D.columns!='label'])))

In [ ]:
feature_importance = {input_cols[i]: clf.feature_importances_[i] for i in range(len(input_cols))}

In [ ]:
feature_importance_sorted = sorted(feature_importance, key=feature_importance.get, reverse=True)

In [ ]:
for r in feature_importance_sorted:
    print('{} : {}'.format(r,feature_importance[r]))

In [ ]:
X.to_csv('jan_x.csv')
! hdfs dfs -put jan_x.csv hdfs://polarisgq-nn1.gq.vcg.yahoo.com:8020/user/kreddy01/call_intet_intl_prediction_experiment/model_object_02_24_2020

In [ ]:
y.to_csv('jan_y.csv',header=True)
! hdfs dfs -put jan_y.csv hdfs://polarisgq-nn1.gq.vcg.yahoo.com:8020/user/kreddy01/call_intet_intl_prediction_experiment/model_object_02_24_2020

In [ ]:
pickle.dump(clf, open( "rf_n_100_d_20.p", "wb" ))
! hdfs dfs -put rf_n_100_d_20.p hdfs://polarisgq-nn1.gq.vcg.yahoo.com:8020/user/kreddy01/call_intet_intl_prediction_experiment/model_object_02_24_2020

In [ ]:
import pickle
import numpy
class model:
    def __init__(self):
        #self.clf=clf
        self.model=pickle.load(open( "rf_n_100_d_20.p", "rb" ) )
    def predict(self,temp):
        self.temp=temp
        if clf.predict(self.temp)==0.0:
            return 'billing'
        elif clf.predict(self.temp)==1.0:
            return 'cruise'
        elif clf.predict(self.temp)==2.0:
            return 'phone_settings'
        elif clf.predict(self.temp)==3.0:
            return 'plan_travelpass'

    def predict_proba(self,temp):
        self.temp=temp
        return np.around(np.max(clf.predict_proba(self.temp)),5)
    def __call__(self,*x_input):
        x_input = list(x_input)[0]
        x_input=np.array(x_input).reshape(-1,1)
        x_predict=self.predict(x_input)
        x_predict_proba=self.predict_proba(x_input)
        return x_predict,x_predict_proba

In [ ]:
class_model=model()
print(class_model.predict(X.tail(1)))
print(class_model.predict_proba(X.tail(1)))

In [ ]:
X.tail(1).to_csv('x_test.csv')
! hdfs dfs -put x_test.csv hdfs://polarisgq-nn1.gq.vcg.yahoo.com:8020/user/vjain01/call_intet_intl_prediction_experiment/model_object_02_24_2020

In [ ]:
#without real time site catalyst columns
